In [29]:
import math
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv("../data/Chembl_bioactivity_SIRT1.tsv", delimiter="\t")

In [23]:
df.columns

Index(['CMPD_CHEMBLID', 'MOLREGNO', 'PARENT_CMPD_CHEMBLID', 'PARENT_MOLREGNO',
       'MOL_PREF_NAME', 'COMPOUND_KEY', 'MOLWEIGHT', 'ALOGP', 'PSA',
       'NUM_RO5_VIOLATIONS', 'CANONICAL_SMILES', 'ACTIVITY_ID',
       'STANDARD_TYPE', 'RELATION', 'STANDARD_VALUE', 'STANDARD_UNITS',
       'PCHEMBL_VALUE', 'PUBLISHED_TYPE', 'PUBLISHED_RELATION',
       'PUBLISHED_VALUE', 'PUBLISHED_UNITS', 'ACTIVITY_COMMENT',
       'DATA_VALIDITY_COMMENT', 'POTENTIAL_DUPLICATE', 'BAO_ENDPOINT',
       'UO_UNITS', 'QUDT_UNITS', 'ASSAY_ID', 'ASSAY_CHEMBLID', 'ASSAY_TYPE',
       'DESCRIPTION', 'ASSAY_SRC_ID', 'ASSAY_SRC_DESCRIPTION',
       'ASSAY_ORGANISM', 'ASSAY_STRAIN', 'ASSAY_TAX_ID', 'CURATED_BY',
       'BAO_FORMAT', 'TID', 'TARGET_CHEMBLID', 'TARGET_TYPE',
       'PROTEIN_ACCESSION', 'PREF_NAME', 'ORGANISM', 'CONFIDENCE_SCORE',
       'TARGET_MAPPING', 'APD_NAME', 'APD_CONFIDENCE', 'DOC_ID',
       'DOC_CHEMBLID', 'PUBMED_ID', 'JOURNAL', 'YEAR', 'VOLUME', 'ISSUE',
       'FIRST_PAGE', 'CELL_ID',

In [24]:
df = df[["CMPD_CHEMBLID", "DOC_CHEMBLID", "CANONICAL_SMILES", "STANDARD_TYPE", "RELATION", "STANDARD_VALUE", "STANDARD_UNITS"]]

In [25]:
df_ic50 = df[df.STANDARD_TYPE == "IC50"]

In [26]:
df_ic50_uniq = df_ic50.drop_duplicates(subset=["CMPD_CHEMBLID", "DOC_CHEMBLID"])

In [27]:
df_ic50_uniq.sort_values(by=["DOC_CHEMBLID", "CMPD_CHEMBLID"])

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS
1884,CHEMBL218785,CHEMBL1137294,CCCCCCCn1cc(C2=C(C(=O)N(C)C2=O)c3cn(CCCCCCC)c4...,IC50,NaN,NaN,NaN
379,CHEMBL291725,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2c[nH]c3ccccc23)c4c[nH]c5cc...,IC50,>,50000.0,nM
1319,CHEMBL296586,CHEMBL1137294,Brc1ccc2[nH]c3c(CC(=O)Nc4ccccc34)c2c1,IC50,NaN,NaN,NaN
614,CHEMBL298346,CHEMBL1137294,O=C1N(Cc2ccccc2)C(=O)C(=C1c3c[nH]c4ccccc34)c5c...,IC50,NaN,NaN,NaN
639,CHEMBL302449,CHEMBL1137294,Cn1c2ccccc2c3c4C(=O)NCc4c5c6ccccc6n(CCC#N)c5c13,IC50,NaN,NaN,NaN
643,CHEMBL380797,CHEMBL1137294,CC(NC(=O)c1ccccc1\N=C\c2c(O)ccc3ccccc23)c4ccccc4,IC50,NaN,NaN,NaN
467,CHEMBL384899,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2cn(Cc3ccccc3)c4ccccc24)c5c...,IC50,NaN,NaN,NaN
425,CHEMBL387119,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2cn(CCCCCCCO)c3ccccc23)c4cn...,IC50,NaN,NaN,NaN
1735,CHEMBL388978,CHEMBL1137294,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n3c4ccccc4...,IC50,NaN,NaN,NaN
103,CHEMBL430574,CHEMBL1137294,O=C1Cc2c([nH]c3ccccc23)c4ccccc4N1,IC50,=,50000.0,nM


In [41]:
def ic50_um_to_pic50(ic50_um):
    return - (np.log10(ic50_um) - 6)

def ic50_to_relevance_score(relation, standard_value, standard_unit, ic50_um_threshold=100):
    """
    relevance_score: large -> good compound
    relation: 
    * "=": pIC50 (ic50_um < ic50_um_threshold) else 0
    * ">": 0 (ic50_um >= ic50_um_threshold) else None
    * "<": pIC50 (ic50_um < ic50_um_threshold) else None
    * "NaN": 0 (the compound may not be inhibitor)
    """
    if standard_unit == "nM":
        ic50_um = standard_value * 1e-3
    elif isinstance(standard_unit, float) and math.isnan(standard_unit):
        return 0
    else:
        return None
    
    if relation == "=":
        if ic50_um < ic50_um_threshold:
            return ic50_um_to_pic50(ic50_um)
        else:
            return 0
    elif relation == ">":
        if ic50_um >= ic50_um_threshold:
            return 0
        else:
            return None
    elif relation == "<":
        if ic50_um < ic50_um_threshold:
            return ic50_um_to_pic50(ic50_um)
        else:
            return None
    else:
        raise ValueError("{} {} {}".format(relation, standard_value, standard_unit))

In [42]:
df_ic50_uniq["relevance"] = df_ic50_uniq.apply(lambda row: ic50_to_relevance_score(row.RELATION, row.STANDARD_VALUE, row.STANDARD_UNITS), axis=1)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
df_ic50_uniq.sort_values(by="relevance", ascending=False)

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance
1357,CHEMBL2332039,CHEMBL2331260,CCNC(=O)c1ccc(s1)C(=O)NCCC2CCN(CC2)c3ncnc4cc(s...,IC50,=,3.6,nM,8.443697
749,CHEMBL2338810,CHEMBL2331260,CS(=O)(=O)NCCC1CCN(CC1)c2ncnc3cc(sc23)C(=O)N,IC50,=,4.3,nM,8.366532
1078,CHEMBL2332037,CHEMBL2331260,CCNC(=O)c1cccc(c1)C(=O)NCCC2CCN(CC2)c3ncnc4cc(...,IC50,=,5.3,nM,8.275724
544,CHEMBL2332043,CHEMBL2331260,NC(=O)c1cc2ncnc(N3CCC(CCNC(=O)c4cccs4)CC3)c2s1,IC50,=,5.8,nM,8.236572
1114,CHEMBL2332041,CHEMBL2331260,CC(C)(C)OC(=O)c1ccc(s1)C(=O)NCCC2CCN(CC2)c3ncn...,IC50,=,6.7,nM,8.173925
1299,CHEMBL2332042,CHEMBL2331260,NC(=O)c1cc2ncnc(N3CCC(CCNC(=O)c4ccc(s4)C(=O)O)...,IC50,=,14.0,nM,7.853872
2070,CHEMBL2332055,CHEMBL2331260,CC(C)(C)C(=O)NCCC1CCN(CC1)c2ncnc3cc(sc23)C(=O)N,IC50,=,15.0,nM,7.823909
884,CHEMBL2332044,CHEMBL2331260,CC(C)(C)OC(=O)NCCC1CCN(CC1)c2ncnc3cc(sc23)C(=O)N,IC50,=,17.0,nM,7.769551
146,CHEMBL2332040,CHEMBL2331260,CCNC(=O)c1ccc(s1)C(=O)NCCN2CCN(CC2)c3ncnc4cc(s...,IC50,=,31.0,nM,7.508638
1320,CHEMBL420311,CHEMBL1795336,NC(=O)C1CCCc2c1[nH]c3ccc(Cl)cc23,IC50,=,38.0,nM,7.420216


In [44]:
df_ic50_uniq.relevance.describe(include="all")

count    669.000000
mean       3.286927
std        2.543104
min        0.000000
25%        0.000000
50%        4.330683
75%        5.050610
max        8.443697
Name: relevance, dtype: float64

In [45]:
df_inhibition = df[df.STANDARD_TYPE == "Inhibition"]

In [46]:
df_inhibition_uniq = df_inhibition.drop_duplicates(subset=["CMPD_CHEMBLID", "DOC_CHEMBLID"])

In [47]:
df_inhibition_uniq.sort_values(by=["DOC_CHEMBLID", "CMPD_CHEMBLID"])

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS
1282,CHEMBL1140,CHEMBL1137294,NC(=O)c1cccnc1,Inhibition,=,10.6,%
1066,CHEMBL218328,CHEMBL1137294,CN1CCCC1CCn2cc(C3=C(C(=O)NC3=O)c4c[nH]c5ccccc4...,Inhibition,=,58.2,%
2028,CHEMBL218910,CHEMBL1137294,OCCC(OCCn1cc(C2=C(C(=O)NC2=O)c3c[nH]c4cccc(F)c...,Inhibition,=,29.9,%
1435,CHEMBL220456,CHEMBL1137294,OCCC(OCCn1cc(C2=C(C(=O)NC2=O)c3c[nH]c4cccc(F)c...,Inhibition,=,71.9,%
2092,CHEMBL263467,CHEMBL1137294,Fc1cccc2c1c3cn2CCO[C@@H](CCn4cc(C5=C3C(=O)NC5=...,Inhibition,=,55.7,%
943,CHEMBL266487,CHEMBL1137294,O=C1NC(=O)C(=C1c2c[nH]c3ccccc23)c4c[nH]c5ccccc45,Inhibition,=,52.7,%
1178,CHEMBL413219,CHEMBL1137294,OCCC(OCCn1cc(C2=C(C(=O)NC2=O)c3c[nH]c4ccccc34)...,Inhibition,=,77.5,%
1967,CHEMBL430574,CHEMBL1137294,O=C1Cc2c([nH]c3ccccc23)c4ccccc4N1,Inhibition,=,49.4,%
1729,CHEMBL443108,CHEMBL1138523,Fc1ccccc1C(=O)C2=Cc3c(OC2=O)ccc4ccccc34,Inhibition,=,3.3,%
504,CHEMBL447256,CHEMBL1138523,Clc1cccc(c1)C(=O)C2=Cc3c(OC2=O)ccc4ccccc34,Inhibition,=,3.7,%


In [48]:
def inhibition_to_relevance_score(relation, standard_value, standard_unit):
    """
    relevance_score: large -> good compound
    relation:
    * "=": max(standard_value, 0)
    * ">": max(standard_value, 0)
    * "<": None
    * "NaN": 0 (the compound may not be inhibitor)
    """
    if standard_unit != "%":
        return None
    elif isinstance(standard_unit, float) and math.isnan(standard_unit):
        return 0
    
    if relation == "=":
        return max(standard_value, 0)
    elif relation == ">":
        return max(standard_value, 0)
    elif relation == "<":
        return None
    else:
        raise ValueError("{} {} {}".format(relation, standard_value, standard_unit))

In [50]:
df_inhibition_uniq["relevance"] = df_inhibition_uniq.apply(lambda row: inhibition_to_relevance_score(row.RELATION, row.STANDARD_VALUE, row.STANDARD_UNITS), axis=1)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [53]:
df_inhibition_uniq.sort_values(by="relevance", ascending=False)

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance
688,CHEMBL2420981,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)CCCCCc1ccccc1)C(=O)NCc2...,Inhibition,=,100.0,%,100.0
1463,CHEMBL1254033,CHEMBL1250463,C[C@H](NC(=O)[C@H](CCCCNC(=[Se])C)NC(=O)[C@H](...,Inhibition,=,100.0,%,100.0
2010,CHEMBL1254613,CHEMBL1250463,CC(C)CC(=S)NCCCC[C@H](NC(=O)[C@H](C)NC(=O)C)C(...,Inhibition,=,100.0,%,100.0
596,CHEMBL1254032,CHEMBL1250463,C[C@H](NC(=O)[C@H](CCCCNC(=S)C)NC(=O)[C@H](C)N...,Inhibition,=,100.0,%,100.0
135,CHEMBL2420982,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)CCCCCCc1ccccc1)C(=O)NCc...,Inhibition,=,99.0,%,99.0
1671,CHEMBL420311,CHEMBL1144627,NC(=O)C1CCCc2c1[nH]c3ccc(Cl)cc23,Inhibition,=,98.9,%,98.9
333,CHEMBL1254035,CHEMBL1250463,CC(=[Se])NCCCC[C@H](NC(=O)OCc1ccccc1)C(=O)Nc2c...,Inhibition,=,98.0,%,98.0
1588,CHEMBL2417693,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)CCCc1ccccc1)C(=O)NCC(=O...,Inhibition,=,98.0,%,98.0
1745,CHEMBL2417694,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)CCCCCc1ccccc1)C(=O)NCC(...,Inhibition,=,98.0,%,98.0
35,CHEMBL1835312,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)OCc1ccccc1)C(=O)NCC(=O)...,Inhibition,=,97.0,%,97.0


In [54]:
df_inhibition_uniq.relevance.describe(include="all")

count    473.000000
mean      36.228985
std       28.444874
min        0.000000
25%       12.000000
50%       29.400000
75%       54.800000
max      100.000000
Name: relevance, dtype: float64

In [55]:
# qid
"""
qid: (SIRT_N, {IC50|INHIBITION}, DOC_ID)
"""
doc_set = set(list(df_ic50_uniq.DOC_CHEMBLID) + list(df_inhibition_uniq.DOC_CHEMBLID))
doc_ind_map = {}
for ind, doc in enumerate(doc_set):
    doc_ind_map[doc] = ind + 1

In [59]:
df_ic50_uniq["qid"] = df_ic50_uniq.apply(lambda row: "1" + "1" + str(doc_ind_map[row.DOC_CHEMBLID]), axis=1)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [64]:
df_ic50_uniq.sort_values(by="DOC_CHEMBLID")

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance,qid
103,CHEMBL430574,CHEMBL1137294,O=C1Cc2c([nH]c3ccccc23)c4ccccc4N1,IC50,=,50000.0,nM,4.301030,113
614,CHEMBL298346,CHEMBL1137294,O=C1N(Cc2ccccc2)C(=O)C(=C1c3c[nH]c4ccccc34)c5c...,IC50,NaN,NaN,NaN,0.000000,113
1884,CHEMBL218785,CHEMBL1137294,CCCCCCCn1cc(C2=C(C(=O)N(C)C2=O)c3cn(CCCCCCC)c4...,IC50,NaN,NaN,NaN,0.000000,113
425,CHEMBL387119,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2cn(CCCCCCCO)c3ccccc23)c4cn...,IC50,NaN,NaN,NaN,0.000000,113
639,CHEMBL302449,CHEMBL1137294,Cn1c2ccccc2c3c4C(=O)NCc4c5c6ccccc6n(CCC#N)c5c13,IC50,NaN,NaN,NaN,0.000000,113
643,CHEMBL380797,CHEMBL1137294,CC(NC(=O)c1ccccc1\N=C\c2c(O)ccc3ccccc23)c4ccccc4,IC50,NaN,NaN,NaN,0.000000,113
1735,CHEMBL388978,CHEMBL1137294,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n3c4ccccc4...,IC50,NaN,NaN,NaN,0.000000,113
1271,CHEMBL98275,CHEMBL1137294,ONC1=Nc2ccccc2c3[nH]c4ccc(Br)cc4c3C1,IC50,NaN,NaN,NaN,0.000000,113
467,CHEMBL384899,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2cn(Cc3ccccc3)c4ccccc24)c5c...,IC50,NaN,NaN,NaN,0.000000,113
379,CHEMBL291725,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2c[nH]c3ccccc23)c4c[nH]c5cc...,IC50,>,50000.0,nM,NaN,113


In [65]:
df_inhibition_uniq["qid"] = df_inhibition_uniq.apply(lambda row: "1" + "2" + str(doc_ind_map[row.DOC_CHEMBLID]), axis=1)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [66]:
df_inhibition_uniq.sort_values(by="qid")

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance,qid
1476,CHEMBL3343493,CHEMBL3351796,Cc1ccc(cc1)C(=O)Nc2cccc3cc(ccc23)S(=O)(=O)O,Inhibition,=,19.0,%,19.0,121
1480,CHEMBL3343508,CHEMBL3351796,Cc1ccc(cc1[N+](=O)[O-])C(=O)Nc2ccc(c3ccccc23)S...,Inhibition,=,23.0,%,23.0,121
795,CHEMBL3343498,CHEMBL3351796,OS(=O)(=O)c1cccc2cccc(NC(=O)c3ccccc3)c12,Inhibition,=,17.0,%,17.0,121
1281,CHEMBL3343514,CHEMBL3351796,NC(=O)c1ccc(Sc2ccc3ccccc3c2)cc1,Inhibition,=,20.0,%,20.0,121
1433,CHEMBL3343513,CHEMBL3351796,NC(=O)c1ccc(Oc2ccc3ccccc3c2)nc1,Inhibition,=,31.0,%,31.0,121
82,CHEMBL3343499,CHEMBL3351796,Cc1ccc(cc1)C(=O)Nc2cccc3cccc(c23)S(=O)(=O)O,Inhibition,<,10.0,%,NaN,121
232,CHEMBL3343504,CHEMBL3351796,OS(=O)(=O)c1ccc(NC(=O)c2ccccc2)c3ccccc13,Inhibition,<,10.0,%,NaN,121
1418,CHEMBL3343510,CHEMBL3351796,NC(=O)c1ccc(Nc2ccc3ccccc3c2)cc1,Inhibition,<,10.0,%,NaN,121
222,CHEMBL3343495,CHEMBL3351796,Nc1cccc(c1)C(=O)Nc2cccc3cc(ccc23)S(=O)(=O)O,Inhibition,<,10.0,%,NaN,121
1906,CHEMBL3343505,CHEMBL3351796,Cc1ccc(cc1)C(=O)Nc2ccc(c3ccccc23)S(=O)(=O)O,Inhibition,=,22.0,%,22.0,121
